In [ ]:
!gdown 16hn7EOuNgWoK3Mgj3LYOy87PQaNJpTL-

Downloading...
From: https://drive.google.com/uc?id=16hn7EOuNgWoK3Mgj3LYOy87PQaNJpTL-
To: /content/SemEval2025_task11.zip
100% 3.40M/3.40M [00:00<00:00, 138MB/s]


In [ ]:
!rm -rf public_data emo_unicode.py
!unzip -q SemEval2025_task11.zip

INSTALLED

In [ ]:
!pip install -q clean-text[gpl]
# !pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import pandas as pd
import time
import os
from sklearn import metrics
import nltk
from nltk.tokenize import word_tokenize
from emo_unicode import UNICODE_EMOJI
from tqdm.notebook import tqdm
from tqdm import tqdm
from cleantext import clean

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**load and preprocess**



constant


In [ ]:
emoji_regex = r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\u2600-\u26FF\u2700-\u27BF]"
labels=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
kinds=["train","dev"]
tasks=["track_a","track_b"]
langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]

In [ ]:


def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    text = re.sub(r"&.*?;",'',text)

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=True,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing
    text = str(text).lower()

    # removing wierd patterns
    wierd_pattern = re.compile("["
        # u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

def convert_emoticons(text):
  emojis=re.findall(emoji_regex,text)
  # return emojis
  for emot in emojis:
    if emot not in UNICODE_EMOJI:
      # print("gosh")
      text = re.sub(u'('+emot+')',"", text)
    else:
      text = re.sub(u'('+emot+')', "_".join(UNICODE_EMOJI[emot].replace(",","").split()), text)
  return text



In [ ]:
def safe_tokenize(message):
    if isinstance(message, str):
        return word_tokenize(message)
    else:
        return []

In [ ]:
tqdm.pandas()
for kind in kinds:
  for task in tasks:
    for lang in langs:
      if kind=="dev":
        lang+="_"+task[-1]
      path=f"public_data/{kind}/{task}/{lang}.csv"
      if not os.path.isfile(path):
        print(path)
        continue
      train_data=pd.read_csv(path)
      train_data['clean_message']=train_data['text'].progress_apply(convert_emoticons).progress_apply(cleaning)
      train_data = train_data.reset_index(drop=True)
      train_data=train_data.drop(columns={"text","id"})
      processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
      directory = os.path.dirname(processed_path)
      if not os.path.exists(directory):
        os.makedirs(directory)

      train_data.to_csv(processed_path)



100%|██████████| 3392/3392 [00:00<00:00, 5380.26it/s]


public_data/train/track_a/sum.csv


100%|██████████| 3681/3681 [00:00<00:00, 4158.85it/s]


public_data/train/track_b/afr.csv


100%|██████████| 2768/2768 [00:00<00:00, 7258.99it/s]


...

public_data/dev/track_b/afr_b.csv


100%|██████████| 116/116 [00:00<00:00, 4628.95it/s]


public_data/dev/track_b/oro_b.csv


100%|██████████| 343/343 [00:00<00:00, 3123.20it/s]

public_data/dev/track_b/som_b.csv
public_data/dev/track_b/sum_b.csv
public_data/dev/track_b/tir_b.csv


In [ ]:
!zip -qr processed_data.zip processed_data

In [ ]:
# train_data[labels].sum()

In [ ]:
# data_counts=train_data[labels].value_counts()
# data_counts

data visualization


In [ ]:
# train_data['num_words']=train_data['clean_message'].str.len()
# fig=plt.figure(figsize=(12,8))
# sns.set_style('dark')

# sns.displot(train_data['num_words'], kde=True,color='Blue')
# mean_val = train_data['num_words'].mean()
# print(f'\nMean:{mean_val:.2f}')
# plt.legend()
# plt.title('number of words in each message',fontsize=16,fontweight='bold')
# plt.xlabel('Number of Words', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)

# plt.grid(True, linestyle='--', alpha=1)
# plt.show()

In [ ]:
# big_len_data= train_data[train_data['clean_message'].apply(lambda x: len(x) >350)]
# big_len_data

In [ ]:
# normal_len_data= train_data[train_data['clean_message'].apply(lambda x: len(x)<=350)]
# normal_len_data